In [ ]:
%%javascript
Jupyter.notebook.session.restart({kernel_name: 'policy-toolkit'})

In [187]:
from PIL import Image
from wand.image import Image as wimage
import os
import io
import pytesseract
import numpy as np
import re

filepath = "../data/raw/Agriculture Rules.pdf"

class Document:
    def __init__(self, path):
        self.path = path
    
    def get_images(self):
        page_images = []
        with wimage(filename=self.path, resolution=200) as img:
            for page_wand_image_seq in img.sequence:
                page_wand_image = wimage(page_wand_image_seq)
                page_jpeg_bytes = page_wand_image.make_blob(format="jpeg")
                page_jpeg_data = io.BytesIO(page_jpeg_bytes)
                page_image = Image.open(page_jpeg_data)
                page_images.append(page_image)
        self.images = page_images
        
    def process_images(self):
        self.text = np.array([pytesseract.image_to_string(x) for x in self.images])
        self.pages = [x for x in range(len(self.images))]
        
    def split_lines(self):
        self.lines = {}
        counter = 0
        pages = [re.split('\n', self.text[i]) for i in self.pages]
        for x, page in enumerate(pages):
            for i, line in enumerate(page):
                self.lines[i + counter] = [line, x]
            counter += len(page)
            
    def combine_sentences(self):
        poplist = []
        max_key = len(self.lines) - 1
        for i in self.lines.keys():
            line = self.lines[i][0]
            if len(line) > 1:
                if line[-1] != '.':
                    self.lines[i][0] = line + " " + self.lines.get(i + 1, ' ')[0]
                    poplist.append(i + 1)
        for x in poplist:
            del self.lines[min(x, max_key)]

In [188]:
doc1 = Document(path = filepath)
doc1.get_images()
doc1.process_images()
doc1.split_lines()
doc1.combine_sentences()